In [ ]:
import mysql.connector

connection = mysql.connector.connect(
            host="crypto-matter.c5eq66ogk1mf.eu-central-1.rds.amazonaws.com",
            user="Nikan",
            passwd="your_password",
            database="Crypto"
        )

cursor = connection.cursor()

unique_symbols = rates_df[['Symbol']].drop_duplicates()['Symbol'].tolist()

existing_symbols_query = "SELECT crypto_symbol FROM crypto_reference WHERE crypto_symbol IN %s"
cursor.execute(existing_symbols_query, (tuple(unique_symbols),))
existing_symbols = {row[0] for row in cursor.fetchall()}

missing_symbols = [(symbol, symbol) for symbol in unique_symbols if symbol not in existing_symbols]

if missing_symbols:
    insert_reference_statement = "INSERT IGNORE INTO crypto_reference (crypto_symbol, crypto_name) VALUES (%s, %s)"
    cursor.executemany(insert_reference_statement, missing_symbols)

insert_statement = """
    INSERT IGNORE INTO crypto_lending_borrowing (crypto_symbol, date, lending_rate, borrowing_rate, utilization_rate, stable_borrow_rate)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

data = rates_df[['Symbol', 'date', 'liquidityRate_avg', 'variableBorrowRate_avg', 'utilizationRate_avg', 'stableBorrowRate_avg']].values.tolist()
cursor.executemany(insert_statement, data)

# Commit and close connection
connection.commit()
cursor.close()
connection.close()